### Human in the loop

In [1]:
from langchain_groq import ChatGroq
import os

import httpx


# llm = init_chat_model("groq:llama3-8b-8192")
# Create SSL-safe Groq client
http_client = httpx.Client(verify=False)
llm = ChatGroq(
    model="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY"),
    http_client=http_client
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1126d8ec0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1126da510>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'), http_client=<httpx.Client object at 0x1123d63c0>)

In [2]:
import urllib3
from langchain_tavily import TavilySearch
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv

load_dotenv()


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


@tool
def human_assistance(query: str) -> str:
    """
    Function to handle human assistance.
    """
    human_response = interrupt({"query": query})
    return human_response["data"]


urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


@tool
def search_tavily(query: str) -> str:
    """Search for information using Tavily API."""
    import requests
    import os

    return requests.post("https://api.tavily.com/search",
                         json={"api_key": os.getenv("TAVILY_API_KEY"),
                               "query": query, "max_results": 3},
                         verify=False).json()


tools = [search_tavily, human_assistance]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

toolNode = ToolNode(tools=tools)
graph_builder.add_node("tools", toolNode)

graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")

In [3]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [4]:
user_input = "I need some expert guidance and assistance in building AI agents. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": user_input},
    config,
    stream_mode="values",
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I need some expert guidance and assistance in building AI agents. Could you request assistance for me?
================================== Ai Message ==================================
Tool Calls:
  human_assistance (yh8pxf9vz)
 Call ID: yh8pxf9vz
  Args:
    query: Building AI agents


In [5]:
human_message = (
    "I can help you with that. We'd recommend starting with the basics of AI agents."
    "It's important to understand the fundamentals before diving into more complex topics."
)

human_command = Command(resume={"data": human_message})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (yh8pxf9vz)
 Call ID: yh8pxf9vz
  Args:
    query: Building AI agents
================================= Tool Message =================================
Name: human_assistance

I can help you with that. We'd recommend starting with the basics of AI agents.It's important to understand the fundamentals before diving into more complex topics.
================================== Ai Message ==================================

Thank you for the tool call result! Based on the response, I can directly respond with the requested guidance:

To build AI agents, it's essential to start with the basics. AI agents typically consist of three main components: perception, reasoning, and action. Perception involves gathering data and information from the environment, reasoning involves processing and analyzing that data to make decisions, and action involves taking actions based on those decision